In [1]:
import pandas as pd
import numpy as np

In [2]:
listSongs = ['Coldplay - The Scientist (Acoustic Cover) (128kbit_AAC)-converted.mp3.csv',
'Coldplay - The Scientist (Cello & Piano) - Brooklyn Duo (152kbit_Opus)-converted.mp3.csv',
'Coldplay - The Scientist (Live in Madrid 2011) (128kbit_AAC)-converted.mp3.csv',
'Coldplay - The Scientist (Official Video) (152kbit_Opus)-converted.mp3.csv',
'Coldplay - The Scientist (Piano Cover) (152kbit_Opus)-converted.mp3.csv',
'Coldplay - The Scientist in the Live Lounge (152kbit_Opus)-converted.mp3.csv',
'Coldplay The Scientist Frankfurt 5-7-22 (128kbit_AAC)-converted.mp3.csv',
'GABRIELLA - Coldplay - The Scientist (Cover) (152kbit_Opus)-converted.mp3.csv',
'GLEE - The Scientist (Full Performance) (Official Music Video) (128kbit_AAC)-converted.mp3.csv',
'GLEE - The Scientist (Full Performance) HD (152kbit_Opus)-converted.mp3.csv',
'Miley Cyrus - The Scientist (Coldplay Cover) (152kbit_Opus)-converted.mp3.csv',
'Savanna Leigh - The Scientist (Lyrics) (152kbit_Opus)-converted.mp3.csv',
'The Scientist (152kbit_Opus)-converted.mp3.csv',
'The Scientist - Coldplay (Boyce Avenue feat. Hannah Trigwell acoustic cover) on Spotify & Apple (152kbit_Opus)-converted.mp3.csv',
'The Scientist - Coldplay (Cover) by Alice Kristiansen (128kbit_AAC)-converted.mp3.csv',
'The Scientist - Coldplay (Holly Henry Acoustic Cover) (152kbit_Opus)-converted.mp3.csv',
'The Scientist - Coldplay (Piano Accompaniment + Tutorial) (152kbit_Opus)-converted.mp3.csv',
'The Scientist - Coldplay Cover (ft. Kina Grannis, Tyler Ward, Lindsey Stirling) (152kbit_Opus)-converted.mp3.csv',
'The Scientist - Coldplay _ Alex Goot & Jada Facer (152kbit_Opus)-converted.mp3.csv',
'The Scientist - The Petersens (LIVE) (128kbit_AAC)-converted.mp3.csv',
'Wicker Park - The Scientist (Coldplay) (192kbit_AAC)-converted.mp3.csv',
'coldplay - the scientist (cover) (128kbit_AAC)-converted.mp3.csv',
'coverTheScientist-converted.mp3.csv',
'gleechapversionthescientist-converted.mp3.csv',
'officialsongtehscientist-converted.mp3.csv',
'piano cover the scientist-converted.mp3.csv',
'thescientistcover-converted.mp3.csv',
'thescientistguitarcover-converted.mp3.csv',
'thescientistliev-converted.mp3.csv']

In [166]:
chordsSongs = []
#Read all the csv from the list and find out the key
for index in range(0, len(listSongs)):
    df = pd.read_csv('../Databases/songs/' + listSongs[index], header=None)
    #Get key from song
    key =  df.value_counts().index[0][0]
    chords = df[0].values.tolist()
    chordsSongs.append((index, key, chords))

In [280]:
#STR Turn b chords to Sharp 
def to_Sharp(chord):
    mapCleanedChords = {
        'Ab': 'G#',
        'Bb': 'A#',
        'Db': 'C#',
        'Eb': 'D#',
        'Gb': 'F#',
        'Abm': 'G#m',
        'Amb': 'G#m',
        'Dbm': 'C#m',
        'Dmb': 'C#m',
        'Gbm': 'F#m',
        'Gmb': 'F#m',
        'Bmb': 'A#m',
        'Bbm': 'A#m',
        'Emb': 'D#m',
        'Ebm': 'D#m',
        'Abdim': 'G#dim',
        'Bbdim': 'A#dim',
        'Dbdim': 'C#dim',
        'Ebdim': 'D#dim',
        'Gbdim': 'F#dim',
    }
    return mapCleanedChords[chord]

#STR Remove all non-accepted chars from a chord
def remove_chars(chord):
    strChord = chord
    strChord = strChord.replace('7', '')
    strChord = strChord.replace('6', '')
    strChord = strChord.replace('5', '')
    strChord = strChord.replace('4', '')
    strChord = strChord.replace('maj', '')
    strChord = strChord.replace('aug', '')    
    strChord = strChord.split('/', 1)[0] if '/' in strChord else strChord
    strChord = to_Sharp(strChord) if 'b' in strChord else strChord
    return strChord

#[] Remove "N"
def remove_nan(chords):
    chordsnotNAN = list(filter(lambda x: x != 'N', chords))
    return chordsnotNAN

#[],[],str,str TRANSPOSE C
def transpose_C(chords, states, song_id, comp_key):
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    chords = remove_nan(chords)
    chords = [remove_chars(chord) for chord in chords]
    new_chords = [""]*len(chords)  # Empty list for new chords
    if comp_key == 'C':
        for chord in chords:
            states.add(chord)
        return(states,
               {'song_id': song_id,
                'orig_key': comp_key,
                'trans_chords': chords})
    else:
        #How many towards "C"
        steps = np.where(np.asarray(notes) == comp_key)[0][0]
        for i in range(len(chords)):
            is_min = False
            is_dim = False
            curr_chord = chords[i]
            if 'm' in curr_chord:
                if "dim" in curr_chord:
                    is_dim = True
                    curr_chord = curr_chord.replace("dim","")
                else:
                    is_min = True
                    curr_chord = curr_chord.replace("m","")
            chord_idx = np.where(np.asarray(notes) == curr_chord)[0][0]
            new_chord_idx = chord_idx - steps
            if is_min:
                new_chords[i] = notes[new_chord_idx] + 'm'
            elif is_dim:
                new_chords[i] = notes[new_chord_idx] + 'dim'
            else:
                new_chords[i] = notes[new_chord_idx]
            states.add(new_chords[i])
        return(states,
               {'song_id': song_id,
                'orig_key': comp_key,
                'trans_chords': new_chords})

#CLEAN CHORDS
def get_clean_chord_progressions(Chords, Id, Key):
    states = set()
    states, songs = transpose_C(Chords, states, Id, Key)
    states = sorted(list(states))
    return(states, songs)

#GET INFO FROM SPECIFIC SONG
def get_song(idx):
    Chords = chordsSongs[idx][2]
    Id = chordsSongs[idx][0]
    Key = remove_chars(chordsSongs[idx][1])
    return (Chords, Id, Key)


#MAIN FUNCTION GET ALL
def transform_whole(idx):
    Chords,Id,Key =  get_song(idx)
    result = get_clean_chord_progressions(Chords ,Id, Key)
    states = result[0]
    songInfo = result[1]
    return(states,songInfo)

def transform_all(info):
    results =[]
    for idx,song in enumerate(info):
        results.append(transform_whole(idx))
    return results

In [ ]:
chordsSongsCleaned = transform_all(chordsSongs)